In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
all_main_run_names={
    "single_regression_counts": "NCCL_P2P_DISABLE=1 python finetuning/train_single_counts_parallel_h5_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 {run_name} /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 2 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed}",
    "single_regression": "NCCL_P2P_DISABLE=1 python finetuning/train_single_parallel_h5_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 {run_name} /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 2 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed}",
    "classification": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_classification_parallel_h5_dataset_dynamic_sampling_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 {run_name} /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 1 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed}",
    "regression": "NCCL_P2P_DISABLE=1 python finetuning/train_pairwise_regression_parallel_h5_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 {run_name} /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 1 --lr 0.0001 --weight_decay 0.001 --use_scheduler --warmup_steps 1000 --data_seed {data_seed}"    
}
all_seeds = [42, 97, 7]

In [3]:
slurm_template = "slurm_template.sh"
temp_script_path = "temp_script.sh"

In [4]:
for run in all_main_run_names:
    for i, seed in enumerate(all_seeds):
        run_name = f"{run}_lr_1e-4_wd_1e-3_replicate{i}"
        cmd = all_main_run_names[run]
        cmd = cmd.replace("{run_name}", run_name)
        cmd = cmd.replace("{data_seed}", str(seed))
        
        print(cmd)
        
        temp_script = open(temp_script_path, "w+")
        for line in open(slurm_template, "r").readlines():
            temp_script.write(line)
        temp_script.write("\n")
        
        temp_script.write(cmd)
        temp_script.write("\n")
        
        temp_script.close()
        
        os.system(f"sbatch {temp_script_path}")

NCCL_P2P_DISABLE=1 python finetuning/train_single_counts_parallel_h5_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 single_regression_counts_lr_5e-4_wd_5e-3_replicate0 /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 2 --lr 0.0005 --weight_decay 0.005 --use_scheduler --warmup_steps 1000 --data_seed 42
Submitted batch job 18142770
NCCL_P2P_DISABLE=1 python finetuning/train_single_counts_parallel_h5_dataset.py /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/train.h5 /clusterfs/nilah/ruchir/src/finetuning-enformer/finetuning/data/h5_bins_384_chrom_split/val.h5 single_regression_counts_lr_5e-4_wd_5e-3_replicate1 /global/scratch/users/aniketh/finetune-enformer/saved_models/ --batch_size 2 --lr 0.0005 --weight_decay 0.005 --use_scheduler --warmup_steps 1000 --data_seed 97
S